In [ ]:
import pandas as pd
from pyprojroot import here

import matplotlib.pyplot as plt
from nutils import colors

In [ ]:
TARGET = 'medical'

In [ ]:
data = pd.read_csv(here() / 'data/preprocessed/targets-h.csv', parse_dates=True, index_col='Datetime')

In [ ]:
y = data[f'Target:Q90_{TARGET}']
y = y[~y.index.weekday.isin([6,7])]

In [ ]:
y.sum() / y.shape[0]

In [ ]:
shift = pd.Series(y.index.hour, index=y.index)
shift = shift.replace(dict.fromkeys([8, 9, 10, 11, 12, 13, 14, 15, 16], 'Morning'))
shift = shift.replace(dict.fromkeys([17, 18, 19, 20, 21, 22, 23], 'Evening'))
shift = shift.replace(dict.fromkeys([0, 1, 2, 3, 4, 5, 6, 7], 'Night'))

In [ ]:
y.groupby(shift).sum() / y.groupby(shift).count()

In [ ]:
mat = y.groupby([y.index.date, y.index.hour]).first().unstack()
a = mat.sum(axis=1).value_counts().sort_index()

b = pd.Series(index=['0 h', '1-2 h', '3-13 h'], data = [a[0], a[1:3].sum(), a[3:].sum()])

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

circle = plt.Circle((0, 0), 
                    radius=0.7,
                    edgecolor='black', 
                    lw=1,
                    facecolor='white',
                    )
  
b.plot.pie(ax=ax, 
           wedgeprops={'edgecolor':'k'},
           colors=[colors['tuni:green'], colors['tuni:yellow'], colors['tuni:red']],
           autopct='%1.0f%%',
           pctdistance=.5
           )

ax.add_patch(circle);

plt.savefig(here() / f'output/plots/pie-{TARGET}.png', dpi=300, bbox_inches='tight')